# Re-level ROSETTA-ice gravity data

import the necessary packages

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz
from antarctic_plots import fetch, regions, utils, maps
import pygmt
import numpy as np
import pandas as pd
import numpy as np
from scipy import stats
import geopandas as gpd
import xarray as xr
import rioxarray
import verde as vd
import harmonica as hm
from pyproj import Transformer
import os
import zarr
import math
import scipy
import warnings
from tqdm.auto import tqdm
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from RIS_gravity_inversion import gravity_processing as grav
from RIS_gravity_inversion import utils as inv_utils

os.getcwd()

## load data
with antarctic_plots.fetch

In [ ]:
rosetta = fetch.ROSETTA_gravity()

# set standard column names
rosetta = rosetta.rename(columns={"Line": "line", "x": "easting", "y": "northing"})

rosetta = gpd.GeoDataFrame(
    rosetta,
    geometry=gpd.points_from_xy(x=rosetta.easting, y=rosetta.northing),
)

rosetta[["FAG_levelled", "Height"]].hist(bins=100)
print(rosetta.describe())

In [ ]:
# seperate e-w flight lines from n-s tie lines
ties = rosetta[rosetta.line >= 1000]
lines = rosetta[rosetta.line < 1000]

Plot as point data

In [ ]:
# get bounding region of data
# region = vd.get_region((rosetta.x, rosetta.y))
# # zoom out by 20 km
# region = utils.alter_region(region, zoom=-40e3)[0]
# # round to nearest 5km
# region = round_region(region, 5e3)
# region
region = (-620000.0, 460000.0, -1460000.0, -380000.0)

In [ ]:
# # plot a basemap
# fig = maps.basemap(
#     region,
#     title="ROSETTA-Ice gravity data",
#     background="black",
# )

# # get a quarter of max absolute value and make a colorscale
# maxabs = vd.maxabs(rosetta.FAG_levelled) * 0.25
# pygmt.makecpt(cmap="roma", series=[-maxabs, maxabs])

# # plot gravity data
# fig.plot(x=rosetta.easting, y=rosetta.northing, color=rosetta.FAG_levelled, style="c2p", cmap=True)

# grav.plot_flightlines(fig, lines, plot_lines=False, direction="EW")

# grav.plot_flightlines(fig, lines, plot_lines=False, direction="NS")

# # plot the colorbar
# fig.colorbar(frame='af+l"free air gravity [mGal]"')

# # add a second figure to the right
# fig = maps.basemap(
#     region,
#     title="ROSETTA-Ice flight elevations",
#     origin_shift="xshift",
#     fig=fig,
#     background="black",
# )

# # make a colorscale for flight elevations
# pygmt.makecpt(cmap="thermal", series=[rosetta.Height.min(), rosetta.Height.max()])

# # plot flight elevation data
# fig.plot(x=rosetta.easting, y=rosetta.northing, color=rosetta.Height, style="c2p", cmap=True)

# grav.plot_flightlines(fig, lines, plot_lines=False, direction="EW")
# grav.plot_flightlines(fig, ties, plot_lines=False, direction="NS")

# # plot the colorbar
# fig.colorbar(frame='af+l"flight elevation [m]"')

# fig.show()

# Intersections
create an intersections dataframe between all lines

In [ ]:
# !!! ONLY MAKES INTERSECTION WITHIN LINE, IF REAL INT IS JUST OUTSIDE END OF LINE
# IT WON'T COUNT. SHOULD BUFFER LINES BY DISTANCE.

# buffer dist extends line ends to include crossovers, in this case, T1040 would intersect with
# L20 if it were just 2 km longer.
inters = grav.create_intersection_table(rosetta, cutoff_dist=10e3, plot=False)
inters.describe()

In [ ]:
# add empty row to
rosetta, inters = grav.add_intersections(rosetta, inters)
rosetta[rosetta.is_intersection]

In [ ]:
inters.describe()

In [ ]:
grav.plotly_points(
    inters,
    color_col="max_dist",
    hover_cols=["line1", "line2", "max_dist"],
    robust=True,
    point_size=6,
    theme=None,
    cmap="greys",
)

In [ ]:
grav.plotly_points(
    rosetta,
    # color_col="intersecting_line",
    color_col="FAG_levelled",
    hover_cols=["line", "dist_along_line", "unixtime"],
    robust=True,
    # point_size=5,
)

In [ ]:
# grav.plotly_points(
#     rosetta[rosetta.line>=1000],
#     color_col="dist_along_line",
#     hover_cols=["line", "dist_along_line", "unixtime"],
#     robust=True,
#     point_size=5,
# )

In [ ]:
# grav.plotly_points(
#     rosetta[rosetta.line<1000],
#     color_col="dist_along_line",
#     hover_cols=["line", "dist_along_line", "unixtime"],
#     robust=True,
# )

In [ ]:
# x = grav.interp1d(
#     rosetta[rosetta.line.isin([1000])],
#     to_interp=["FAG_levelled", "Height"],

#     # interp_on=["x","y"],
#     # engine="verde",
#     # method=vd.Spline(),

#     interp_on="dist_along_line",
#     engine="scipy",
#     method="cubic",

#     # plot_line=True,
# )

# # x[x.is_intersection]

In [ ]:
# x = grav.interp1d_windows(
#     rosetta[rosetta.line.isin([1000])],
#     to_interp=["FAG_levelled", "Height"],

#     window_width = 1e3,

#     # interp_on=["x","y"],
#     # engine="verde",
#     # method=vd.Spline(),

#     interp_on="dist_along_line",
#     engine="scipy",
#     method="cubic",

#     # plot_windows=True,
#     # plot_line=True,
# )

# # x[x.is_intersection]

## Interpolate data at intersections

In [ ]:
rosetta = grav.interp1d_all_lines(
    rosetta,
    # rosetta[rosetta.line.isin(lines_with_elev_issues)],
    # rosetta[rosetta.line.between(1080, 1100)],
    # rosetta[rosetta.line==630],
    to_interp=["FAG_levelled", "Height"],
    window_width=5e3,
    # interp_on=["x","y"],
    # engine="verde",
    # method=vd.Spline(),
    interp_on="dist_along_line",
    engine="scipy",
    method="cubic",
    # plot=True,
    # wait_for_input=True,
)

## Manually check and clean data

In [ ]:
lines_with_no_intersections = [
    20,
    560.0,
]
lines_with_int_issues = [
    250,  # gap at 1030
    430,  # gap at 1100
    470,  # iterpolated grav at 1050, 1040, and 1030
    510,  # gap at 1000
    630,  # gap at 1100, 1120
    660,  # gap at 1020
    680,  # gap at 1090 # might be close enough
    700,  # gap at 1020, 1050
    720,  # gap at 1020
    1020,  # gap at 660,
]
intersection_issues = [
    [250, 1300],
    [430, 1100],
    # [470, 1030],
    [470, 1040],
    # [470, 1050],
    [510, 1000],
    [630, 1100],
    [630, 1120],
    [660, 1020],
    [680, 1090],
    [700, 1020],
    [700, 1050],
    [720, 1020],
    [1020, 660],
]

lines_with_elev_issues = [
    590,  # short chunk of data at start
]
lines_with_grav_issues = [
    450,  # flat at both ends
    470,  # flat at start, flat between ints of 1050 and 1030, very negative at start
    590,  # short chunk of data at start
    870,  # flat at end
    900,  # flat at start
    910,  # flat at start
    1070,  # flat at end
    1120,  # very negative at start
    1140,  # very positive at start
]

In [ ]:
# grav.plot_line_and_crosses(
#     rosetta_inters,
#     line=1140,
#     y=["FAG_levelled", "Height"],
#     y_axes=[1,2],
# )

In [ ]:
df = rosetta.copy()
to_remove = []

# 450: delete flat grav data at start
clean = df.loc[
    (df.line == 450) & ((df.dist_along_line < 6e3) | (df.dist_along_line > 802e3))
]
print(f"removing {len(clean)} points from line {clean.line.unique()}")
to_remove.append(clean)

# 470: delete flat grav data at start and flat data between ints 1030 and 1050
clean = df.loc[
    (df.line == 470)
    & ((df.dist_along_line < 57e3) | (df.dist_along_line.between(476.5e3, 585.5e3)))
]
print(f"removing {len(clean)} points from line {clean.line.unique()}")
to_remove.append(clean)

# 590: delete grav + height data at start
clean = df.loc[(df.line == 590) & (df.dist_along_line < 10e3)]
print(f"removing {len(clean)} points from line {clean.line.unique()}")
to_remove.append(clean)

# 870: delete grav + height data at end
clean = df.loc[(df.line == 870) & (df.dist_along_line > 318e3)]
print(f"removing {len(clean)} points from line {clean.line.unique()}")
to_remove.append(clean)

# 900: delete grav + height data at start
clean = df.loc[(df.line == 900) & (df.dist_along_line < 3.8e3)]
print(f"removing {len(clean)} points from line {clean.line.unique()}")
to_remove.append(clean)

# 910: delete grav + height data at start
clean = df.loc[(df.line == 910) & (df.dist_along_line < 26e3)]
print(f"removing {len(clean)} points from line {clean.line.unique()}")
to_remove.append(clean)

# 1070: delete grav + height data at end, removes intersection with 880
clean = df.loc[(df.line == 1070) & (df.dist_along_line > 700e3)]
print(f"removing {len(clean)} points from line {clean.line.unique()}")
to_remove.append(clean)

df_remove = pd.concat(to_remove)

rosetta_cleaned = rosetta.drop(index=df_remove.index)
assert len(rosetta_cleaned) == len(rosetta) - len(df_remove)

print(f"\ntotal number of rows in dataframe: {len(rosetta_cleaned)}")

In [ ]:
print(
    f"number of intersections in dataframe: {len(rosetta_cleaned[rosetta_cleaned.is_intersection])}"
)
print(f"number of intersections: {len(inters)}")

## Re-do intersections with cleaned data

In [ ]:
inters = grav.create_intersection_table(rosetta_cleaned, cutoff_dist=10e3, plot=False)
rosetta_df, inters = grav.add_intersections(rosetta_cleaned, inters)

print(
    f"number of intersections in dataframe: {len(rosetta_df[rosetta_df.is_intersection])}"
)
print(f"number of intersections: {len(inters)}")

## Re-interpolate at new intersections

In [ ]:
rosetta_df = grav.interp1d_all_lines(
    rosetta_df,
    to_interp=["FAG_levelled", "Height"],
    window_width=5e3,
    interp_on="dist_along_line",
    engine="scipy",
    method="cubic",
    # plot=True,
    # wait_for_input=True,
)
rosetta_df[rosetta_df.is_intersection]

##  Get crossover errors

https://gist.github.com/maptastik/dc3d3b4514546310500a13fb77663bb9


In [ ]:
inters, rosetta_df = grav.calculate_misties(
    inters,
    rosetta_df,
    data_col="FAG_levelled",
    mistie_name="mistie_before_UC",
    # plot=True,
    robust=False,
)
print(
    f"mistie RMSE / mean: {utils.RMSE(inters.mistie_before_UC)} / {np.nanmean(inters.mistie_before_UC)}"
)
inters.sort_values(by="mistie_before_UC")

## Save / load from csv

In [ ]:
# rosetta_df.to_csv(
#     "../data/ROSETTA_cleaned.csv.gz",
#     sep=",",
#     na_rep="",
#     header=True,
#     index=False,
#     encoding="utf-8",
#     compression="gzip",
# )
# inters.to_file(
#      "../data/ROSETTA_cleaned_intersections.gpkg",
# )

In [ ]:
rosetta_df = pd.read_csv(
    "../data/ROSETTA_cleaned.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)
inters = gpd.GeoDataFrame.from_file(
    "../data/ROSETTA_cleaned_intersections.gpkg",
)
rosetta_df

# Individually block-reduce lines

In [ ]:
grouped = rosetta_df.groupby("line")

blocked_dfs = []
for name, group in grouped:
    df = group.drop(columns="geometry")

    blocked = utils.block_reduce(
        df.drop(columns=["is_intersection", "intersecting_line"]),
        np.mean,
        spacing=1e3,
        center_coordinates=False,
        input_coord_names=["easting", "northing"],
    )

    # add back columns
    blocked["is_intersection"] = False
    blocked["intersecting_line"] = ""

    # merge back intersection points
    prior_len = len(blocked)
    blocked = pd.concat([blocked, df[df.is_intersection]])
    assert len(blocked) == prior_len + len(df[df.is_intersection])

    blocked_dfs.append(blocked)

rosetta_blocked = pd.concat(blocked_dfs)
rosetta_blocked

In [ ]:
rosetta_blocked[rosetta_blocked.is_intersection]

## Save / load from csv

In [ ]:
# rosetta_blocked.to_csv(
#     "../data/ROSETTA_cleaned_blocked.csv.gz",
#     sep=",",
#     na_rep="",
#     header=True,
#     index=False,
#     encoding="utf-8",
#     compression="gzip",
# )

In [ ]:
rosetta_blocked = pd.read_csv(
    "../data/ROSETTA_cleaned_blocked.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)
rosetta_blocked

In [ ]:
grav.plotly_points(
    rosetta_blocked,
    coord_names=["easting", "northing"],
    color_col="FAG_levelled",
    hover_cols=["FAG_levelled", "Height", "line"],
    robust=True,
    # point_size=4,
)

In [ ]:
grav.plotly_points(
    rosetta_blocked,
    coord_names=["easting", "northing"],
    color_col="Height",
    hover_cols=["FAG_levelled", "Height", "line"],
    robust=True,
    # point_size=5,
)

# Individually upward continue lines

## find optimal parameters for each line

In [ ]:
df = rosetta_blocked.copy()
lines = df.groupby("line")

damping_limits = [1, 1000]
depth_limits = [0, 50e3]

best_dampings = []
best_depths = []
eqs_bests = []
line_names = []
scores = []
for line, df in tqdm(lines, desc="Lines"):
    coords = (df.easting, df.northing, df.Height)
    data = df.FAG_levelled

    study_df, eqs = inv_utils.optimize_eq_source_params(
        coords,
        data,
        n_trials=10,
        damping_limits=damping_limits,
        depth_limits=depth_limits,
        parallel=False,
        # fname="tmp",
        # use_existing=True,
        plot=False,
    )

    # get best score
    score = study_df.iloc[0].value

    # add values to lists
    best_dampings.append(eqs.damping)
    best_depths.append(eqs.depth)
    eqs_bests.append(eqs)
    line_names.append(line)
    scores.append(score)

best_eqs_per_line = pd.DataFrame(
    {
        "line": line_names,
        "eq_fitted": eqs_bests,
        "damping": best_dampings,
        "depth": best_depths,
        "score": scores,
    }
)

best_eqs_per_line.to_pickle("../data/RIS_best_eqs_per_line.pkl")

best_eqs_per_line = pd.read_pickle(
    "../data/RIS_best_eqs_per_line.pkl",
)

print(
    f"Dampings: possible range: {damping_limits[0], damping_limits[1]}\n",
    f"  min: {np.min(best_eqs_per_line.damping)}\n",
    f"  mode: {best_eqs_per_line.damping.mode()[0]}\n",
    f"  max: {np.max(best_eqs_per_line.damping)}",
)
print(
    f"Depths: possible range: {depth_limits[0], depth_limits[1]}\n",
    f"  min: {np.min(best_eqs_per_line.depth)}\n",
    f"  mode: {best_eqs_per_line.depth.mode()[0]}\n",
    f"  max: {np.max(best_eqs_per_line.depth)}",
)
best_eqs_per_line

In [ ]:
df = best_eqs_per_line
# df.sort_values(by="dampings", inplace=True)

best = df.score.argmax()

sns.set_theme()

fig, axes = plt.subplots(1, 2, figsize=(10, 3.5))
plt.suptitle(f"Equivalent Source Parameters")

# Dampings
axes[0].scatter(
    df.damping,
    df.score,
    marker="o",
)
axes[0].scatter(
    df.damping.iloc[best],
    df.score.iloc[best],
    marker="s",
    # markersize=10,
    color=sns.color_palette()[3],
    label="Minimum",
)
axes[0].legend(loc="best")
axes[0].set_xlabel("Damping")
axes[0].set_ylabel("Score")

# Depths
axes[1].scatter(
    df.depth,
    df.score,
    marker="o",
)
axes[1].scatter(
    df.depth.iloc[best],
    df.score.iloc[best],
    marker="s",
    # markersize=10,
    color=sns.color_palette()[3],
    label="Minimum",
)
axes[1].set_xlabel("Depth")
axes[1].set_ylabel("Score")
axes[1].legend(loc="best")

ylims = [0.995, 1.0005]
axes[0].set_ylim(ylims)
axes[1].set_ylim(ylims)

plt.tight_layout()

## Upward continue data 

In [ ]:
plot = False

# df = rosetta_blocked[rosetta_blocked.line.isin(best_eqs_per_line.line.unique())].copy()

grouped = rosetta_blocked.copy()
grouped["up_cont_grav"] = np.nan
grouped = grouped.groupby("line")

line_numbers = []
up_cont_RMSE = []
line_dfs = []
for line, df in tqdm(grouped, desc="Lines"):
    eq_fitted = best_eqs_per_line[best_eqs_per_line.line == line].eq_fitted.iloc[0]

    # up cont to 1km at some points as before
    values = eq_fitted.predict(
        (df.easting, df.northing, np.ones_like(df.northing) * 1e3)
    )
    df.loc[df.line == line, "up_cont_grav"] = values

    if plot is True:
        grav.plot_line_and_crosses(
            df,
            line=line,
            y=["Height", "FAG_levelled", "up_cont_grav"],
        )

    # compare to original gravity
    df["dif"] = df.up_cont_grav - df.FAG_levelled

    rmse = utils.RMSE(df[df.line == line].dif)

    line_numbers.append(line)
    up_cont_RMSE.append(rmse)
    line_dfs.append(df)

rosetta_UC = pd.concat(line_dfs)

eq_diffs = pd.DataFrame(
    {"line": line_numbers, "up_cont_RMSE": up_cont_RMSE}
).sort_values(by="up_cont_RMSE", ascending=False)
eq_diffs

## Update the misties

In [ ]:
print(f"Previous mistie RMSE: {utils.RMSE(inters.mistie_before_UC)}")
inters, rosetta_UC = grav.calculate_misties(
    inters,
    rosetta_UC,
    data_col="up_cont_grav",
    mistie_name="mistie_after_UC",
    # plot=True,
    robust=False,
)
print(f"Updated mistie RMSE: {utils.RMSE(inters.mistie_after_UC)}")
inters.sort_values(by="mistie_after_UC").head()

## Save and load from csv

In [ ]:
rosetta_UC.to_csv(
    "../data/ROSETTA_UC.csv.gz",
    sep=",",
    na_rep="",
    header=True,
    index=False,
    encoding="utf-8",
    compression="gzip",
)
inters.to_file(
    "../data/ROSETTA_UC_intersections.gpkg",
)

In [ ]:
rosetta_UC = pd.read_csv(
    "../data/ROSETTA_UC.csv.gz",
    sep=",",
    header="infer",
    index_col=None,
    compression="gzip",
)
inters = gpd.GeoDataFrame.from_file(
    "../data/ROSETTA_UC_intersections.gpkg",
)
inters

In [ ]:
grav.plotly_points(
    rosetta_UC,
    color_col="dif",
    hover_cols=["FAG_levelled", "up_cont_grav", "dif", "Height", "line"],
    robust=True,
    # point_size=5,
)

In [ ]:
grav.plotly_points(
    rosetta_UC,
    color_col="FAG_levelled",
    hover_cols=["FAG_levelled", "up_cont_grav", "dif", "Height", "line"],
    robust=True,
    # point_size=5,
)

In [ ]:
grav.plotly_points(
    rosetta_UC,
    color_col="up_cont_grav",
    hover_cols=["FAG_levelled", "up_cont_grav", "dif", "Height", "line"],
    robust=True,
    # point_size=5,
)

In [ ]:
# grav.plot_line_and_crosses(
#     df,
#     line=80,
#     y=["Height", "FAG_levelled", "up_cont_grav",],
# )

# Level lines to ties

## 0th order levelling
Apply DC-shifts to all lines to minimize mistie values

calculates least-squares trend through intersection points

In [ ]:
# # level an individual line
# line = 1090
# df1 = grav.verde_predict_trend(
#     inters[inters.line2==line],
#     ["easting", "northing", "mistie_after_UC"],
#     rosetta_UC[rosetta_UC.line==line],
#     ["easting", "northing", "levelling_correction"],
#     degree=1,
# )
# df1['levelled'] = df1.up_cont_grav - df1.levelling_correction

# grav.plot_line_and_crosses(
#     df1,
#     y=[
#         "up_cont_grav",
#         "levelled",
#         "levelling_correction",
#         "mistie_after_UC"
#     ],
#     y_axes = [1,1,1,1],
#     plot_inters = [True, False, False, False],
#     marker_sizes=[2,2,2,10]
#     )

In [ ]:
# # level an individual line
# line = 190
# df1 = grav.skl_predict_trend(
#     inters[(inters.line1==line) | (inters.line2==line)],
#     ["dist_along_line1", "mistie_after_UC"],
#     rosetta_UC[rosetta_UC.line==line],
#     ["dist_along_line", "levelling_correction"],
#     degree=1,
# )
# df1['levelled'] = df1.up_cont_grav - df1.levelling_correction

# grav.plot_line_and_crosses(
#     df1,
#     y=[
#         "up_cont_grav",
#         "levelled",
#         "levelling_correction",
#         "mistie_after_UC"
#     ],
#     y_axes = [1,1,1,1],
#     plot_inters = [True, False, False, False],
#     marker_sizes=[2,2,2,10]
#     )

In [ ]:
# # level a group of lines
# rosetta_UC, inters = grav.level_lines(
#     inters,
#     rosetta_UC,
#     lines_to_level = rosetta_UC.line[rosetta_UC.line<1000].unique(),
#     cols_to_fit = "dist_along_line1",
#     cols_to_predict = "dist_along_line",
#     degree=0,
#     data_col = "up_cont_grav",
#     mistie_col = "mistie_before_UC",
#     new_mistie_col = "mistie_trend0_1",
#     levelled_col = "levelled",
#     plot=True,
# )

In [ ]:
# perform iterative levelling, just moving lines to match ties
rosetta_trend0, inters = grav.iterative_line_levelling(
    inters,
    rosetta_UC,
    flight_line_names=rosetta_UC.line[rosetta_UC.line < 1000].unique(),
    degree=0,
    starting_mistie_col="mistie_after_UC",
    starting_data_col="up_cont_grav",
    cols_to_fit="dist_along_line1",
    iterations=5,
    # plot_iterations=True,
    plot_results=True,
)
trend0_final_mistie_col = list(rosetta_trend0.columns)[-1]
trend0_final_data_col = list(rosetta_trend0.columns)[-2]

print(trend0_final_mistie_col)
print(trend0_final_data_col)

rosetta_trend0.describe()

## 1st order levelling

In [ ]:
# repeate at a higher order trend
rosetta_trend1, inters = grav.iterative_line_levelling(
    inters,
    rosetta_trend0,
    flight_line_names=rosetta_trend0.line[rosetta_trend0.line < 1000].unique(),
    degree=1,
    starting_mistie_col=trend0_final_mistie_col,
    starting_data_col=trend0_final_data_col,
    cols_to_fit="dist_along_line1",
    iterations=5,
    # plot_iterations=True,
    plot_results=True,
)
trend1_final_mistie_col = list(rosetta_trend1.columns)[-1]
trend1_final_data_col = list(rosetta_trend1.columns)[-2]

print(trend1_final_mistie_col)
print(trend1_final_data_col)

rosetta_trend1.describe()

In [ ]:
# grav.plotly_points(
#     rosetta_trend1,
#     color_col=trend1_final_data_col,
#     hover_cols=["FAG_levelled", "up_cont_grav", "dif", "Height", "line"],
#     robust=True,
#     # point_size=5,
# )

## 2nd order levelling

In [ ]:
# repeate at a higher order trend
rosetta_trend2, inters = grav.iterative_line_levelling(
    inters,
    rosetta_trend1,
    flight_line_names=rosetta_trend1.line[rosetta_trend1.line < 1000].unique(),
    degree=2,
    starting_mistie_col=trend1_final_mistie_col,
    starting_data_col=trend1_final_data_col,
    cols_to_fit="dist_along_line1",
    iterations=5,
    # plot_iterations=True,
    plot_results=True,
)
trend2_final_mistie_col = list(rosetta_trend2.columns)[-1]
trend2_final_data_col = list(rosetta_trend2.columns)[-2]

print(trend2_final_mistie_col)
print(trend2_final_data_col)

rosetta_trend2.describe()

In [ ]:
grav.plotly_points(
    rosetta_trend2,
    color_col=trend2_final_data_col,
    hover_cols=["FAG_levelled", "up_cont_grav", "dif", "Height", "line"],
    robust=True,
    # point_size=5,
)

In [ ]:
eqs = hm.EquivalentSources(damping=1e2, depth=10e3, block_size=2e3)

coords = (
    rosetta_trend2.easting,
    rosetta_trend2.northing,
    np.ones_like(rosetta_trend2.northing) * 1e3,
)

data = rosetta_trend2[trend2_final_data_col]

eqs.fit(coords, data)

In [ ]:
# Define grid coordinates
region = vd.pad_region(vd.get_region(coords), 20e3)
grid_coords = vd.grid_coordinates(
    region=region,
    spacing=5e3,
    extra_coords=1e3,  # upward continue to 1km
)

grid = eqs.grid(grid_coords)

masked = vd.distance_mask((coords[0], coords[1]), maxdist=20e3, grid=grid)
masked = masked.reset_coords(names="upward")
masked.scalars.plot(robust=True)

# Level ties to lines

## 0th order

In [ ]:
rosetta_ties_trend0, inters = grav.iterative_line_levelling(
    inters,
    rosetta_trend2,
    flight_line_names=rosetta_trend2.line[rosetta_trend2.line >= 1000].unique(),
    degree=0,
    starting_mistie_col=trend2_final_mistie_col,
    starting_data_col=trend2_final_data_col,
    iterations=5,
    mistie_prefix="ties_to_lines_mistie_trend0",
    levelled_data_prefix="ties_to_lines_levelled_data_trend0",
    cols_to_fit="dist_along_line2",
    # plot_iterations=True,
    plot_results=True,
)
ties_trend0_final_mistie_col = list(rosetta_ties_trend0.columns)[-1]
ties_trend0_final_data_col = list(rosetta_ties_trend0.columns)[-2]

print(ties_trend0_final_mistie_col)
print(ties_trend0_final_data_col)

rosetta_ties_trend0.describe()

In [ ]:
ties_trend0_final_mistie_col

In [ ]:
ties_trend0_final_data_col

In [ ]:
grav.plotly_points(
    rosetta_ties_trend0,
    color_col=ties_trend0_final_data_col,
    hover_cols=["FAG_levelled", "up_cont_grav", "dif", "Height", "line"],
    robust=True,
    # point_size=5,
)

## 1st order

In [ ]:
rosetta_ties_trend1, inters = grav.iterative_line_levelling(
    inters,
    rosetta_ties_trend0,
    flight_line_names=rosetta_ties_trend0.line[
        rosetta_ties_trend0.line >= 1000
    ].unique(),
    degree=1,
    starting_mistie_col=ties_trend0_final_mistie_col,
    starting_data_col=ties_trend0_final_data_col,
    mistie_prefix="ties_to_lines_mistie_trend1",
    levelled_data_prefix="ties_to_lines_levelled_data_trend1",
    cols_to_fit="dist_along_line2",
    iterations=5,
    # plot_iterations=True,
    plot_results=True,
)
ties_trend1_final_mistie_col = list(rosetta_ties_trend1.columns)[-1]
ties_trend1_final_data_col = list(rosetta_ties_trend1.columns)[-2]

rosetta_ties_trend1.describe()

In [ ]:
ties_trend1_final_mistie_col

In [ ]:
ties_trend1_final_data_col

In [ ]:
grav.plotly_points(
    rosetta_ties_trend1,
    color_col=ties_trend1_final_data_col,
    hover_cols=["FAG_levelled", "up_cont_grav", "dif", "Height", "line"],
    robust=True,
    # point_size=5,
)

In [ ]:
# perform iterative leveling, alternating between flight and tie lines
rosetta_UC, inters = grav.iterative_levelling_alternate(
    inters,
    rosetta_UC,
    tie_line_names=rosetta_UC.line[rosetta_UC.line >= 1000].unique(),
    flight_line_names=rosetta_UC.line[rosetta_UC.line < 1000].unique(),
    degree=0,
    starting_mistie_col="mistie_after_UC",
    starting_data_col="up_cont_grav",
    iterations=4,
    # plot_iterations=True,
    plot_results=True,
)

In [ ]:
rosetta_UC, inters = grav.level_lines(
    inters,
    rosetta_UC,
    lines_to_level=rosetta_UC.line[rosetta_UC.line > 1000].unique(),
    cols_to_fit="dist_along_line2",
    cols_to_predict="dist_along_line",
    degree=0,
    data_col="levelled",
    mistie_col="mistie_3",
    new_mistie_col="mistie_4",
    levelled_col="levelled2",
    plot=True,
)

In [ ]:
rosetta_UC, inters = grav.level_lines(
    inters,
    rosetta_UC,
    lines_to_level=rosetta_UC.line[rosetta_UC.line < 1000].unique(),
    cols_to_fit="dist_along_line1",
    cols_to_predict="dist_along_line",
    degree=0,
    data_col="levelled2",
    mistie_col="mistie_4",
    new_mistie_col="mistie_5",
    levelled_col="levelled3",
    plot=True,
)

In [ ]:
# iterate through the E-W lines

rosetta_UC["levelled"] = np.nan

for line in rosetta_UC[rosetta_UC.line < 1000].line.unique():
    df = rosetta_UC[rosetta_UC.line == line].copy()

    # fit a polynomial trend through the misties values
    ints = inters[inters.line1 == line]
    coords = (ints.geometry.x, ints.geometry.y)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="Under-determined problem")
        try:
            trend = vd.Trend(degree=0).fit(coords, ints.mistie_2)
            # predict the trend on the line data
            df["trend"] = trend.predict((df.easting, df.northing))
        except ValueError:
            print(f"Issue with line {line}")
            df["trend"] = 0

    # remove the trend from the gravity
    values = df.up_cont_grav - df.trend

    rosetta_UC.loc[rosetta_UC.line == line, "levelled"] = values

for line in rosetta_UC[rosetta_UC.line >= 1000].line.unique():
    rosetta_UC.loc[rosetta_UC.line == line, "levelled"] = rosetta_UC.loc[
        rosetta_UC.line == line, "up_cont_grav"
    ]

rosetta_UC["levelling_change"] = rosetta_UC.up_cont_grav - rosetta_UC.levelled

In [ ]:
print(f"Previous mistie RMSE: {utils.RMSE(inters.mistie_2)}")
inters = grav.calculate_misties(
    inters,
    rosetta_UC,
    data_col="levelled",
    mistie_name="mistie_3",
    # plot=True,
    robust=False,
)
inters.sort_values(by="mistie_3")

grav.plotly_points(rosetta_UC, color_col="up_cont_grav", point_size=2)
grav.plotly_points(rosetta_UC, color_col="levelled", point_size=2)
grav.plotly_points(rosetta_UC, color_col="levelling_change", point_size=2)
grav.plotly_points(inters, color_col="mistie_2", point_size=4)
grav.plotly_points(inters, color_col="mistie_3", point_size=4)

In [ ]:
# fit a polynomial trend through the misties
coords = (inters.geometry.x, inters.geometry.y)
trend = vd.Trend(degree=20).fit(coords, inters.mistie_2)

# predict the trend on the intersection points
inters["trend"] = trend.predict(coords)

# remove the trend from the misties
inters["residual"] = inters.mistie_2 - inters.trend

print(utils.RMSE(inters.mistie_2), inters.mistie_2.mean())
print(utils.RMSE(inters.residual), inters.residual.mean())

grav.plotly_points(rosetta_UC, color_col="up_cont_grav", point_size=2)
grav.plotly_points(inters, color_col="mistie_2", point_size=4)
grav.plotly_points(inters, color_col="trend", point_size=4)
grav.plotly_points(inters, color_col="residual", point_size=4)

In [ ]:
# apply correction to the line data

# fit a polynomial trend to the crossover misties
trend = vd.Trend(degree=2).fit(coords, inters.mistie_2)

# predict that trend on the flight line datapoints
rosetta_UC["trend"] = trend.predict((rosetta_UC.easting, rosetta_UC.northing))

# remove the trend from the data
rosetta_UC["levelled"] = rosetta_UC.up_cont_grav - rosetta_UC.trend

grav.plotly_points(
    rosetta_UC,
    color_col="up_cont_grav",
    point_size=2,
    hover_cols=["up_cont_grav", "levelled", "trend"],
)
grav.plotly_points(
    rosetta_UC,
    color_col="trend",
    point_size=2,
    hover_cols=["up_cont_grav", "levelled", "trend"],
)
grav.plotly_points(
    rosetta_UC,
    color_col="levelled",
    point_size=2,
    hover_cols=["up_cont_grav", "levelled", "trend"],
)

In [ ]:
inters.drop(columns=["line1_value", "line2_value"], inplace=True)
inters

In [ ]:
print(f"Previous mistie RMSE: {utils.RMSE(inters.mistie_2)}")
inters = grav.calculate_misties(
    inters,
    rosetta_UC,
    data_col="levelled",
    mistie_name="mistie_3",
    plot=True,
    robust=False,
)
inters.sort_values(by="mistie_3")

## 1st order levelling
Tilt lines to minimize misfit values